## imports voor het project:
* glob: gebruiken we voor het ophalen van gegevens van bestanden
* os: gebruiken we om de current directory op te halen en te veranderen
* csv: gebruiken we voor het ophalen van informatie uit csvbestanden
* sqlite3: gebruiken we vooor het maken/gebruiken van databases

In [2]:
import glob
import os
import csv
import sqlite3

## verander de current directory naar de directory waar de data staat:

In [3]:
os.chdir(os.getcwd()+"/NO_XXX_EVENT_EXPORT")

## stel alle globale variabelen in:

In [4]:
brugnummers = ('W5','WA','WB','WBH','WD','WE','WG','WGR','WH','WHE','WHO','WHT','WHY','WL','WO','WOB','WS','WT','WW','ZBS','ZST')
jaartallen = ('2015','2016','2017','2018')

## geef een overzicht van het aantal bestanden per brug:

In [5]:
print("overzicht van het aantal bestanden van "+str(len(brugnummers))+" bruggen:")
for i in brugnummers:
    x=0
    for file in glob.glob('*'+i+'_*'):
        x += 1
    print("  "+str(x)+" bestanden gevonden voor brug "+ i)

overzicht van het aantal bestanden van 21 bruggen:
  1115 bestanden gevonden voor brug W5
  1098 bestanden gevonden voor brug WA
  1110 bestanden gevonden voor brug WB
  1110 bestanden gevonden voor brug WBH
  1097 bestanden gevonden voor brug WD
  1110 bestanden gevonden voor brug WE
  1115 bestanden gevonden voor brug WG
  1110 bestanden gevonden voor brug WGR
  1110 bestanden gevonden voor brug WH
  1110 bestanden gevonden voor brug WHE
  1110 bestanden gevonden voor brug WHO
  1114 bestanden gevonden voor brug WHT
  1115 bestanden gevonden voor brug WHY
  1110 bestanden gevonden voor brug WL
  1113 bestanden gevonden voor brug WO
  1111 bestanden gevonden voor brug WOB
  1097 bestanden gevonden voor brug WS
  1110 bestanden gevonden voor brug WT
  1110 bestanden gevonden voor brug WW
  1115 bestanden gevonden voor brug ZBS
  1084 bestanden gevonden voor brug ZST


## geef een overzicht van het aantal bestanden per jaartal:

In [6]:
print("overzicht van het aantal bestanden van "+str(len(jaartallen))+" jaren:")
for i in jaartallen:
    x=0
    for file in glob.glob('*'+i+'*'):
        x += 1
    print("  "+str(x)+" bestanden gevonden voor jaar "+ i)

overzicht van het aantal bestanden van 4 jaren:
  621 bestanden gevonden voor jaar 2015
  7777 bestanden gevonden voor jaar 2016
  7761 bestanden gevonden voor jaar 2017
  7115 bestanden gevonden voor jaar 2018


## geef het aantal dubbele logs weer (2 logs op 1 dag):

In [7]:
print("overzicht van het aantal bestanden van dubbele logs per dag:")
x=0
for file in glob.glob('*_2.csv'):
    x += 1
print("    "+str(x)+" bestanden gevonden voor dubbele logs per dag")

overzicht van het aantal bestanden van dubbele logs per dag:
    267 bestanden gevonden voor dubbele logs per dag


## geef een overzicht van het aantal bestanden per dag zonder de dubbele logs:

In [8]:
print("overzicht van het aantal enkele logs van "+str(len(brugnummers))+" bruggen:")
for i in brugnummers:
    x=0
    for file in glob.glob('*'+i+'_*'):
        x += 1
    for file in glob.glob('*'+i+'*_2.csv'):
        x -= 1    
    print("  "+str(x)+" bestanden gevonden voor brug "+ i+" zonder dubbele logs")

overzicht van het aantal enkele logs van 21 bruggen:
  1097 bestanden gevonden voor brug W5 zonder dubbele logs
  1097 bestanden gevonden voor brug WA zonder dubbele logs
  1084 bestanden gevonden voor brug WB zonder dubbele logs
  1097 bestanden gevonden voor brug WBH zonder dubbele logs
  1097 bestanden gevonden voor brug WD zonder dubbele logs
  1097 bestanden gevonden voor brug WE zonder dubbele logs
  1084 bestanden gevonden voor brug WG zonder dubbele logs
  1097 bestanden gevonden voor brug WGR zonder dubbele logs
  1036 bestanden gevonden voor brug WH zonder dubbele logs
  1097 bestanden gevonden voor brug WHE zonder dubbele logs
  1097 bestanden gevonden voor brug WHO zonder dubbele logs
  1097 bestanden gevonden voor brug WHT zonder dubbele logs
  1097 bestanden gevonden voor brug WHY zonder dubbele logs
  1097 bestanden gevonden voor brug WL zonder dubbele logs
  1083 bestanden gevonden voor brug WO zonder dubbele logs
  1097 bestanden gevonden voor brug WOB zonder dubbele l

## schrijf alle csvfiles naar de database (bridges.db):

In [10]:
if os.path.isfile('bridges.db'):
    os.remove("bridges.db")

con = sqlite3.connect("bridges.db")
cur = con.cursor()

for x in brugnummers:
    cur.execute("CREATE TABLE IF NOT EXISTS "+x+" (Tijd text, Type text, Klasse text, Waarde text, ID text, Omschrijving text, Status text, Prioriteit text, Comment text, Tijd_Came text, Tijd_Went text, Tijd_ack text, UUID text, DPE text);")
    print("tabel "+x+ " is succesvol aangemaakt!")
    for file in glob.glob('*'+x+'_*'):
        with open(file) as csv_file:
            reader = csv.reader(csv_file,  delimiter=";")
            for i in range(24):
                try:
                    next(reader)
                except:
                    print(csv_file.name+" heeft niet het goede format!")
                    break
            for i in reader:
                to_db = (i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7],i[8],i[9],i[10],i[11],i[12],i[13])
                cur.executemany("INSERT INTO "+x+" (Tijd, Type, Klasse, Waarde, ID, Omschrijving, Status, Prioriteit, Comment, Tijd_Came, Tijd_Went, Tijd_ack, UUID, DPE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", (to_db,))
        con.commit()
        print("  "+csv_file.name+" is in de database gezet!")
    print("brug "+x+" is volledig in de database gezet!")
con.close()
print("alle csv files zijn nu naar de database bridges.db geschreven!")


tabel W5 is succesvol aangemaakt!
  NO_W5_EVENT_EXPORT_2015-12-01_00-00-00_2015-12-01_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-02_00-00-00_2015-12-02_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-03_00-00-00_2015-12-03_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-04_00-00-00_2015-12-04_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-05_00-00-00_2015-12-05_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-06_00-00-00_2015-12-06_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-07_00-00-00_2015-12-07_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-08_00-00-00_2015-12-08_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-09_00-00-00_2015-12-09_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-10_00-00-00_2015-12-10_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2015-12-11_00-00-00_2015-12-11_23-59-59.CSV i

  NO_W5_EVENT_EXPORT_2016-02-29_00-00-00_2016-02-29_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-01_00-00-00_2016-03-01_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-02_00-00-00_2016-03-02_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-03_00-00-00_2016-03-03_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-04_00-00-00_2016-03-04_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-05_00-00-00_2016-03-05_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-06_00-00-00_2016-03-06_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-07_00-00-00_2016-03-07_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-08_00-00-00_2016-03-08_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-09_00-00-00_2016-03-09_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-03-10_00-00-00_2016-03-10_23-59-59.CSV is in de database gezet!
  NO_W5_EV

  NO_W5_EVENT_EXPORT_2016-05-30_00-00-00_2016-05-30_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-05-31_00-00-00_2016-05-31_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-01_00-00-00_2016-06-01_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-01_00-00-00_2016-06-01_23-59-59_2.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-02_00-00-00_2016-06-02_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-03_00-00-00_2016-06-03_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-04_00-00-00_2016-06-04_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-05_00-00-00_2016-06-05_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-06_00-00-00_2016-06-06_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-07_00-00-00_2016-06-07_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-06-08_00-00-00_2016-06-08_23-59-59.CSV is in de database gezet!
  NO_W5_

  NO_W5_EVENT_EXPORT_2016-08-29_00-00-00_2016-08-29_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-08-30_00-00-00_2016-08-30_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-08-31_00-00-00_2016-08-31_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-01_00-00-00_2016-09-01_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-01_00-00-00_2016-09-01_23-59-59_2.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-02_00-00-00_2016-09-02_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-03_00-00-00_2016-09-03_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-04_00-00-00_2016-09-04_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-05_00-00-00_2016-09-05_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-06_00-00-00_2016-09-06_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-09-07_00-00-00_2016-09-07_23-59-59.CSV is in de database gezet!
  NO_W5_

  NO_W5_EVENT_EXPORT_2016-11-27_00-00-00_2016-11-27_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-11-28_00-00-00_2016-11-28_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-11-29_00-00-00_2016-11-29_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-11-30_00-00-00_2016-11-30_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-12-01_00-00-00_2016-12-01_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-12-02_00-00-00_2016-12-02_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-12-03_00-00-00_2016-12-03_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-12-04_00-00-00_2016-12-04_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-12-05_00-00-00_2016-12-05_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-12-06_00-00-00_2016-12-06_23-59-59.CSV is in de database gezet!
  NO_W5_EVENT_EXPORT_2016-12-07_00-00-00_2016-12-07_23-59-59.CSV is in de database gezet!
  NO_W5_EV

...

  NO_ZST_EVENT_EXPORT_2017-09-14_00-00-00_2017-09-14_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-15_00-00-00_2017-09-15_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-16_00-00-00_2017-09-16_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-17_00-00-00_2017-09-17_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-18_00-00-00_2017-09-18_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-19_00-00-00_2017-09-19_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-20_00-00-00_2017-09-20_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-21_00-00-00_2017-09-21_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-22_00-00-00_2017-09-22_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-23_00-00-00_2017-09-23_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-09-24_00-00-00_2017-09-24_23-59-59.CSV is in de database gezet!

  NO_ZST_EVENT_EXPORT_2017-12-13_00-00-00_2017-12-13_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-14_00-00-00_2017-12-14_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-15_00-00-00_2017-12-15_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-16_00-00-00_2017-12-16_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-17_00-00-00_2017-12-17_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-18_00-00-00_2017-12-18_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-19_00-00-00_2017-12-19_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-20_00-00-00_2017-12-20_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-21_00-00-00_2017-12-21_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-22_00-00-00_2017-12-22_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2017-12-23_00-00-00_2017-12-23_23-59-59.CSV is in de database gezet!

  NO_ZST_EVENT_EXPORT_2018-03-14_00-00-00_2018-03-14_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-15_00-00-00_2018-03-15_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-16_00-00-00_2018-03-16_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-17_00-00-00_2018-03-17_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-18_00-00-00_2018-03-18_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-19_00-00-00_2018-03-19_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-20_00-00-00_2018-03-20_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-21_00-00-00_2018-03-21_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-22_00-00-00_2018-03-22_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-23_00-00-00_2018-03-23_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-03-24_00-00-00_2018-03-24_23-59-59.CSV is in de database gezet!

  NO_ZST_EVENT_EXPORT_2018-06-10_00-00-00_2018-06-10_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-11_00-00-00_2018-06-11_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-12_00-00-00_2018-06-12_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-13_00-00-00_2018-06-13_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-14_00-00-00_2018-06-14_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-15_00-00-00_2018-06-15_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-16_00-00-00_2018-06-16_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-17_00-00-00_2018-06-17_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-18_00-00-00_2018-06-18_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-19_00-00-00_2018-06-19_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-06-20_00-00-00_2018-06-20_23-59-59.CSV is in de database gezet!

  NO_ZST_EVENT_EXPORT_2018-09-06_00-00-00_2018-09-06_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-07_00-00-00_2018-09-07_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-08_00-00-00_2018-09-08_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-09_00-00-00_2018-09-09_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-10_00-00-00_2018-09-10_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-11_00-00-00_2018-09-11_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-12_00-00-00_2018-09-12_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-13_00-00-00_2018-09-13_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-14_00-00-00_2018-09-14_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-15_00-00-00_2018-09-15_23-59-59.CSV is in de database gezet!
  NO_ZST_EVENT_EXPORT_2018-09-16_00-00-00_2018-09-16_23-59-59.CSV is in de database gezet!